### IMPORTS

In [1]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az

$\textbf{QUESTION 1}$

In [2]:
babies = pd.read_csv('babies.csv')
cord_clamped = babies['x']
not_clamped = babies['y']

In [3]:
print(cord_clamped.describe())

count    16.000000
mean      9.643750
std       1.714631
min       8.000000
25%       8.350000
50%       9.150000
75%      10.300000
max      13.800000
Name: x, dtype: float64


In [4]:
print(not_clamped.describe())

count    16.00000
mean     12.09375
std       2.23591
min       8.20000
25%      11.00000
50%      12.05000
75%      13.52500
max      16.20000
Name: y, dtype: float64


In [5]:

with pm.Model() as model:

    # Define noninformative Gamma priors
    shape_prior = 0.001
    rate_prior = 0.001
    alpha1 = pm.Gamma(name="alpha1", alpha=shape_prior, beta=rate_prior)
    beta1 = pm.Gamma(name="beta1", alpha=shape_prior, beta=rate_prior)
    alpha2 = pm.Gamma(name="alpha2", alpha=shape_prior, beta=rate_prior)
    beta2 = pm.Gamma(name="beta2", alpha=shape_prior, beta=rate_prior)

    # Priors for the means
    # mean1 = pm.Gamma(name='mean1', alpha=shape_prior, beta=rate_prior)
    # mean2 = pm.Gamma(name='mean2', alpha=shape_prior, beta=rate_prior)
    mean1, mean2 = alpha1 / beta1, alpha2 / beta2

    # Likelihoods for the data
    likelihood1 = pm.Gamma(name='likelihood1', alpha=alpha1, beta=beta1, observed=cord_clamped)
    likelihood2 = pm.Gamma(name='likelihood2', alpha=alpha2, beta=beta2, observed=not_clamped)

    # Difference in means
    diff = mean1 - mean2
    diff_means = pm.Deterministic(name='diff_means', var=diff)

    # Sampling
    trace = pm.sample(draws=1000, tune=1000, target_accept=0.90, cores=None, chains=4)

In [6]:
# View trace diff_means
diff_means_trace = az.summary(data=trace, var_names=['diff_means'], hdi_prob=.90)

# Print trace summary
print(f"Trace Summary:\n{az.summary(data=trace, hdi_prob=.90)}\n")

# Check Credible Set
print(f"The 90% Credible Set for the difference of means:\n{diff_means_trace}")

Trace Summary:
              mean      sd  hdi_5%  hdi_95%  mcse_mean  mcse_sd  ess_bulk  \
alpha1      35.240  12.667  14.512   54.020      0.363    0.258    1182.0   
beta1        3.656   1.324   1.529    5.648      0.038    0.027    1193.0   
alpha2      27.824  10.282  12.252   44.303      0.317    0.224    1021.0   
beta2        2.299   0.855   0.958    3.614      0.027    0.019    1013.0   
diff_means  -2.478   0.766  -3.713   -1.207      0.012    0.009    4085.0   

            ess_tail  r_hat  
alpha1        1106.0    1.0  
beta1         1201.0    1.0  
alpha2        1142.0    1.0  
beta2         1140.0    1.0  
diff_means    2649.0    1.0  

The 90% Credible Set for the difference of means:
             mean     sd  hdi_5%  hdi_95%  mcse_mean  mcse_sd  ess_bulk  \
diff_means -2.478  0.766  -3.713   -1.207      0.012    0.009    4085.0   

            ess_tail  r_hat  
diff_means    2649.0    1.0  


$$
\text{The 90\% credible set doesn't contain 0.} \Rightarrow \text{The difference is statistically significant.}
$$

$\textbf{QUESTION 2}$

In [7]:
intraocular_pressure = pd.read_excel('iop2.xlsx', header=None, names=['indicator', 'cornea_thickness'])
low_iop = intraocular_pressure['indicator']
corn_thickness = intraocular_pressure['cornea_thickness']
corn_mean = corn_thickness.mean()
corn_std = corn_thickness.std()

In [8]:
print(low_iop.describe())

count    140.000000
mean       0.242857
std        0.430349
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: indicator, dtype: float64


In [9]:
print(corn_thickness.describe())

count    140.000000
mean     484.657143
std       39.032319
min      386.000000
25%      456.750000
50%      482.500000
75%      513.000000
max      590.000000
Name: cornea_thickness, dtype: float64


In [10]:
# Borrow some code from Aaron's GitHub
def standardize(x, mu, sig):
        return (x - mu) / (2 * sig)

$$
\textbf{PART A}\\~\\
\text{Model With Logit}
$$

In [11]:
# Borrow some code from Aaron's GitHub
with pm.Model() as mod_logistic:

    # Define x, y
    corn_standard_log = standardize(x=corn_thickness,
                                mu=corn_mean,
                                sig=corn_std)
    corn_log = pm.Data(name="corn_data",
                        value=corn_standard_log,
                        mutable=True)
    iop_log = pm.Data(name="iop_data",
                       value=low_iop,
                       mutable=False)

    # Define alpha, beta for logistic regression
    alpha = pm.Normal(name="alpha",
                      mu=0,
                      sigma=2)
    betas = pm.Normal(name="beta",
                      mu=0,
                      sigma=1)

    logist = alpha + pm.math.dot(l=corn_log,
                                 r=betas)
    p = pm.math.invlogit(logist)

    pm.Bernoulli(name="low_iop",
                 p=p,
                 observed=iop_log)

    trace_log = pm.sample(draws=1000,
                          tune=1000,
                          cores=None,
                          chains=4,
                          idata_kwargs=dict(log_likelihood=True))

$$
\textbf{PART B}\\~\\
\text{Model With Logit and X = 490}
$$

In [12]:
# Borrow some code from Aaron's GitHub
with pm.Model() as mod_logistic490:

    # Define x, y
    corn_log490 = standardize(x=490,
                            mu=corn_mean,
                            sig=corn_std)

    iop_log490 = pm.Data(name="iop_data",
                       value=low_iop,
                       mutable=False)

    # Define alpha, beta for logistic regression
    alpha = pm.Normal(name="alpha",
                      mu=0,
                      sigma=2)
    betas = pm.Normal(name="beta",
                      mu=0,
                      sigma=1)

    logist = alpha + pm.math.dot(l=corn_log490,
                                 r=betas)
    p = pm.math.invlogit(logist)

    pm.Bernoulli(name="low_iop",
                p=p,
                observed=iop_log490)

    trace_log490 = pm.sample(draws=1000,
                             tune=1000,
                             cores=None,
                             chains=4,
                             idata_kwargs=dict(log_likelihood=True))

In [13]:
with mod_logistic490:

    # Get Predictions
    preds_log490 = pm.sample_posterior_predictive(trace=trace_log490,
                                                predictions=True)
az.summary(data=preds_log490.predictions).mean()

mean            0.245257
sd              0.430193
hdi_3%          0.000000
hdi_97%         1.000000
mcse_mean       0.006986
mcse_sd         0.005000
ess_bulk     3931.571429
ess_tail     3897.771429
r_hat           1.000000
dtype: float64

$$
\textbf{PART C}\\~\\
\text{Model With Probit}
$$

In [14]:
# Borrow some code from Aaron's GitHub
with pm.Model() as mod_probit:

    # Define x, y
    corn_standard_prob = standardize(x=corn_thickness,
                                mu=corn_mean,
                                sig=corn_std)
    corn_prob = pm.Data(name="corn_data",
                        value=corn_standard_prob,
                        mutable=True)
    iop_prob = pm.Data(name="iop_data",
                        value=low_iop,
                        mutable=False)

    # Define alpha, beta for logistic regression
    alpha = pm.Normal(name="alpha",
                      mu=0,
                      sigma=2)
    betas = pm.Normal(name="beta",
                      mu=0,
                      sigma=1)

    logist = alpha + pm.math.dot(l=corn_prob,
                                 r=betas)
    p = pm.math.invprobit(logist)

    pm.Bernoulli(name="low_iop",
                 p=p,
                 observed=iop_prob)

    trace_prob = pm.sample(draws=1000,
                           tune=1000,
                           cores=None,
                           chains=4,
                           idata_kwargs=dict(log_likelihood=True))

$$
\text{Find Deviances:}
$$

In [15]:
with mod_probit:

    # View Deviances
    dev_prob = az.waic(data=trace_prob,
                  scale='deviance')
    print(f"Deviance for Probit:\n{dev_prob}")

with mod_logistic:

    # View Deviances
    dev_logit = az.waic(data=trace_log,
                        scale='deviance')
    print(f"Deviance for Logit:\n{dev_logit}")

Deviance for Probit:
Computed from 4000 posterior samples and 140 observations log-likelihood matrix.

              Estimate       SE
deviance_waic   135.81    12.02
p_waic            1.69        -
Deviance for Logit:
Computed from 4000 posterior samples and 140 observations log-likelihood matrix.

              Estimate       SE
deviance_waic   136.67    11.47
p_waic            1.52        -


$\textbf{QUESTION 3}$

In [16]:
micronuclei = pd.read_csv('micronuclei.csv')
rad_dose = micronuclei['x']
freq = micronuclei['y']

In [17]:
rad_mean = rad_dose.mean()
rad_std = rad_dose.std()

In [18]:
print(freq.describe())

count    6000.000000
mean        0.303000
std         0.634501
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
Name: y, dtype: float64


$$
\textbf{PART A}
$$

In [19]:
with pm.Model() as mod_poisson:

    rad_pois_stand = standardize(x=rad_dose,
                            mu=rad_mean,
                            sig=rad_std)

    rad_pois = pm.Data(name="rad_pois",
                       value=rad_pois_stand,
                       mutable=True)

    freq_pois = pm.Data(name="freq_pois",
                   value=freq,
                   mutable=True)

    beta0 = pm.Normal("beta0_intercept", mu=0, tau=0.0001)
    beta1 = pm.Normal('beta1', mu=0, tau=0.0001)

    regression = beta0 + pm.math.dot(l=rad_pois, r=beta1)
    mu = pm.math.exp(regression)

    y = pm.Poisson('y', mu=mu, observed=freq_pois)

    trace_pois = pm.sample(draws=1000, tune=1000, cores=None, chains=4)

$$
\textbf{PART B}
$$

In [22]:
# Borrow some code from Aaron's GitHub
with pm.Model() as mod_pois25:

    # Define x, y
    rad_pois_standard = standardize(x=2.5,
                            mu=rad_mean,
                            sig=rad_std)

    rad_pois25 = pm.Data(name="rad_pois25",
                        value=rad_pois_standard,
                        mutable=True)

    freq_pois25 = pm.Data(name="freq_pois25",
                    value=freq,
                    mutable=True)

    beta0 = pm.Normal("beta0_intercept", mu=0, tau=0.0001)
    beta1 = pm.Normal('beta1', mu=0, tau=0.0001)

    regression = beta0 + pm.math.dot(l=rad_pois25, r=beta1)
    mu = pm.math.exp(regression)

    y = pm.Poisson('y', mu=mu, observed=freq_pois25)

    trace_pois25 = pm.sample(draws=2000, tune=1000, cores=None, chains=4)

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
ERROR:pymc.stats.convergence:There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


In [23]:
with mod_pois25:

    # Get Predictions
    preds_pois25 = pm.sample_posterior_predictive(trace=trace_pois,
                                                predictions=True)
az.summary(data=preds_pois25.predictions).mean()


mean            0.321161
sd              0.566852
hdi_3%          0.000000
hdi_97%         1.000000
mcse_mean       0.009035
mcse_sd         0.006308
ess_bulk     3942.875333
ess_tail     3851.567333
r_hat           1.000000
dtype: float64